In [6]:
import pandas as pd
import numpy as np

In [46]:
from enum import Enum

In [144]:
from functools import wraps

In [179]:
class Permission(Enum):
    Admin = 1
    Student = 2
    
    def require(permission: Permission):
        def wrapper(f):
            @wraps(f)
            def wrapped(self, *args, **kwargs):
                if self.session and self.session.permission == permission:
                    return f(self, *args, **kwargs)
                raise Exception("Invalid Permission")
            return wrapped
        return wrapper

In [172]:
class T(Enum):
    contacts = 1
    students = 2
    classes = 3
    course = 4
    credits = 5

In [173]:
class Session(object):
    def __init__(self, user: int, permission: Permission):
        self.user = user
        self.permission = permission

    def isAdmin(self):
        return self.permission == Permission.admin

In [174]:
files = {
    T.contacts: 'contacts.csv',
    T.students: 'students.csv',
    T.classes: 'class.csv',
    T.course: 'course.csv',
    T.credits: 'credits.csv',
}

In [ ]:

def strip(text):
    try:
        return text.strip()
    except AttributeError:
        return text

def make_int(text):
    return int(text.strip('" '))

table = pd.read_table("data.csv", sep=r',',
                      names=["Year", "Make", "Model", "Description"],
                      converters = {'Description' : strip,
                                    'Model' : strip,
                                    'Make' : strip,
                                    'Year' : make_int})

In [203]:
Manager.converter[T.contacts]('   123   ')

'123'

In [374]:
class Manager:
    converter = {t: lambda x: x.strip() for t in T}
    
    def __init__(self, files):
        if not all(t in files for t in T):
            raise "Must include all files"
        self.frames = {file: pd.read_csv(filename, dtype=object, sep=" *, *") for file, filename in files.items()}
        self.columns = {file: frame.columns.values for file, frame in self.frames.items()}
        self.values = {file: frame.values for file, frame in self.frames.items()}
        self.session = None
    
    def login(self, user, pw):
        if user == 'admin':
            self.session = Session(user, Permission.Admin)
    
    @Permission.require(Permission.Admin)
    def create(self, target, values):
        self.values[target] = np.vstack([self.values[target], np.array(values)])
    
    @Permission.require(Permission.Admin)
    def remove(self, target, index):
        pass
    
    @Permission.require(Permission.Admin)
    def update(self, target, where, values):
        pass
    
    def find(self, subject, target, where):
        return db.values[target][np.where(
            np.logical_and(
                *[db.values[target][:,column] == value for column, value in zip(
                    [np.where(db.columns[target] == case)[0][0] for case in where],
                    where.values()
                )], True
            )
        )][:, [np.where(db.columns[target] == case)[0][0] for case in (subject if subject else db.columns[target])]]
    
    def select(self, target, index):
        return self.values[np.where(self.raw[:,0] == 2013004394)].squeeze()
    
    def __str__(self):
        pass

In [375]:
db = Manager(files)

/Users/user/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


In [357]:
db.login('admin', None)

In [358]:
db.create(T.students, [2016001234, 'xxx', '홍길동', 'male', 6, 1999002345, 1])

In [359]:
db.create(T.contacts, [2016001234, '01088884444', 'hong@hanyang.ac.kr'])

In [365]:
db.update(T.contacts, {
    "sid": db.find(['sid'], T.students, {'sname': '권희조'}).squeeze()[0]
}, {
    "email": "kwon@hanyang.ac.kr"
})

IndexError: too many indices for array

In [364]:
db.columns[T.contacts]

array(['sid', 'phone', 'email'], dtype=object)